<a href="https://colab.research.google.com/github/NachoC20/RiesgoMercado/blob/main/VaR_CVaR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%load_ext rpy2.ipython
!rm -rf RiesgosFinancieros
!git clone "https://github.com/pikey-msc/RiesgosFinancieros/"

Cloning into 'RiesgosFinancieros'...
remote: Enumerating objects: 691, done.
remote: Counting objects: 100% (112/112), done.
remote: Compressing objects: 100% (112/112), done.
remote: Total 691 (delta 77), reused 0 (delta 0), pack-reused 579
Receiving objects: 100% (691/691), 40.95 MiB | 21.95 MiB/s, done.
Resolving deltas: 100% (456/456), done.


# Tarea 1: Riesgo de Mercado

In [ ]:
%%R
#Instalamos los paquetes que necesitaremos.
install.packages("quantmod")
install.packages("data.table")
install.packages("PerformanceAnalytics")
install.packages("Deriv")

#Cargamos librerias
library(Deriv)
library(quantmod)
library(data.table)
library("PerformanceAnalytics")
library(dplyr)

con = gzcon(url('https://github.com/systematicinvestor/SIT/raw/master/sit.gz', 'rb'))
source(con)
close(con)

R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: also installing the dependencies ‘xts’, ‘zoo’, ‘TTR’


R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/xts_0.12.1.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 517875 bytes (505 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[

In [401]:

%%R
## GENERAMOS LAS FUNCIONES QUE NECESITAREMOS:

# Función de interpolación de tasas por el método alamabrada
talamb=function(nodos,curva,plazos){
  n=max(ncol(plazos),1)
  m=max(ncol(nodos),1)
  TC=matrix(0,1,n)
  TL=matrix(0,1,n)
  TF=matrix(0,1,n)
  for (j in 1:n){
    i=1
    repeat
    {
      if(nodos[i]<= plazos[j] && plazos[j] <=nodos[i+1])
      {
        TC[j]=curva[i]
        TL[j]=curva[i+1]
        TF[j]=((((1+TL[j]*nodos[i+1]/360)/(1+TC[j]*nodos[i]/360))^((plazos[j]-nodos[i])/(nodos[i+1]-nodos[i]))*(1+TC[j]*nodos[i]/360))-1)*360/plazos[j]
        break
      }
      else if (plazos[j]<nodos[1])
      {
        TC[j]=curva[1]
        TL[j]=curva[1]
        TF[j]=curva[1]
        break
      }
      else if (plazos[j]>nodos[m])
      {
        TC[j]=curva[m]
        TL[j]=curva[m]
        TF[j]=curva[m]
        break
      }
      else
      {i=i+1}
    }
  }
  as.matrix(t(as.numeric(rbind(TF))))
}

# Función para diagonalizar un vector
diagv=function(x)	{
  n01=nrow(as.matrix(x))
  m01=ncol(as.matrix(x))
  dimmax=max(n01,m01)
  res=matrix(0,dimmax,dimmax)
  for (i in 1:dimmax){
    res[i,i]=x[i]
  }
  res
}

#Función de cuantil más cercano
equantile <- function(v,p=.5,ns=nrow(as.matrix(v))) 
{ 
  if ( !is.numeric(p) || any( p<0 | p>1) ) 
    stop("Percentil tiene que ser 0<=p<=1") 
  ranking <- order(v) 
  vw=matrix(0,ns,1)
  vw[1:ns]=seq(1/ns,ns)  
  sumw <- cumsum(vw[ranking]) 
  plist <- sumw / sumw[ length(sumw) ] 
  v [ ranking [ which.max( plist >= p ) ] ]  
} 


wquantile <- function(v,w=rep(1,length(v)),p=.5) 
{ 
  if ( !is.numeric(w) || length(v) != length(w) ) 
    stop("Los valores y los pesos tienen que tener misma longitud") 
  if ( !is.numeric(p) || any( p<0 | p>1) ) 
    stop("Percentil tiene que ser 0<=p<=1") 
  if ( min(w) < 0 ) stop("Los pesos tiene que ser mayores que 0") 
  ranking <- order(v) 
  sumw <- cumsum(w[ranking]) 
  plist <- sumw / sumw[ length(sumw) ] 
  v [ ranking [ which.max( plist >= p ) ] ]  
}

#CVaR con alisado
wcvar <- function(v,w=rep(1,length(v)),p=.5) 
{ 
  if ( !is.numeric(w) || length(v) != length(w) ) 
    stop("Los valores y los pesos tienen que tener misma longitud") 
  if ( !is.numeric(p) || any( p<0 | p>1) ) 
    stop("Percentil tiene que ser 0<=p<=1") 
  if ( min(w) < 0 ) stop("Los pesos tiene que ser mayores que 0") 
  ranking <- order(v) 
  sumw <- cumsum(w[ranking]) 
  plist <- sumw / sumw[ length(sumw) ] 
  loss= v [ ranking [ which( plist < p ) ] ]  
  esc=w [ ranking [ which( plist < p ) ] ]  
  sum(loss*esc)/(sum(esc))
}

## Para el alisado

In [402]:
%%R
w0 <- 0.05 #@param {type:"number"}

In [ ]:
%%R
lambda =uniroot(function(x) w0*(1-x^(n))/(1-x)-1, c(0,0.99), tol = 1e-28)$root

#generamos la función que genera "n" escenarios con base en w0 y lambda
genera_esc=function(lamda,w0,n)
{
  p_esc=matrix(0,n,1)
  for (i in (1:n))
  {
    p_esc[i]=w0*lambda^(i-1)
  }
  p_esc
}

In [403]:
%%R
#VaR con alisado

wquantile <- function(v,w=rep(1,length(v)),p=.5) 
{ 
  if ( !is.numeric(w) || length(v) != length(w) ) 
    stop("Los valores y los pesos tienen que tener misma longitud") 
  if ( !is.numeric(p) || any( p<0 | p>1) ) 
    stop("Percentil tiene que ser 0<=p<=1") 
  if ( min(w) < 0 ) stop("Los pesos tiene que ser mayores que 0") 
  ranking <- order(v) 
  sumw <- cumsum(w[ranking]) 
  plist <- sumw / sumw[ length(sumw) ] 
  v [ ranking [ which.max( plist >= p ) ] ]  
} 

#CVaR con alisado
wcvar <- function(v,w=rep(1,length(v)),p=.5) 
{ 
  if ( !is.numeric(w) || length(v) != length(w) ) 
    stop("Los valores y los pesos tienen que tener misma longitud") 
  if ( !is.numeric(p) || any( p<0 | p>1) ) 
    stop("Percentil tiene que ser 0<=p<=1") 
  if ( min(w) < 0 ) stop("Los pesos tiene que ser mayores que 0") 
  ranking <- order(v) 
  sumw <- cumsum(w[ranking]) 
  plist <- sumw / sumw[ length(sumw) ] 
  loss= v [ ranking [ which( plist <= p ) ] ]  
  esc=w [ ranking [ which( plist <= p ) ] ]  
  sum(loss*esc)/(sum(esc))
} 


## **Definición de parámetros para valorar**

In [404]:
%%R

fval=as.Date("20200313",format="%Y%m%d") #Fecha de valoración
itpl=0 #poner 0 si se quiere interpolación lineal o 1 si se quiere tasa alambrada
alpha=0.98 #Nivel de confianza para obtener estimaciones de riesgo

### Inciso 1
1000 acciones de GCarso, -5000 de AméricaMóvil y 1200 WalMart. *Todas de la BMV*

In [405]:
%%R
Symbols<-c  ("AMXL.MX", "GCARSOA1.MX", "WALMEX.MX" )#Claves de cotización de las emisoras, en orden alfabético
pos_eq=c(-5000,1000,1200) #Monto inicial invertido en acciones
nh=3660 #Días de historia

### Inciso 2
1500 dólares (peso dólar), 700 euros, y -600 libras esterlinas.

In [406]:
%%R
SymbolsFX<-c("EURUSD=X","GBPUSD=X","USDMXN=X" ) #Simbolos en orden alfabetico
pos_fx=c(700,-600,1500) #Monto incial invertido en divisas
nh=3660 #días de historia


### Inciso 3
2 bonos largos, uno cupón cero de cetes (con la curva guber) con nocional de 1500 vencimiento de 180 días, un bono M con tasa fija de 4.5% con un cupon de 4.05% anual vencimiento de 3600 días y un bono corto 1000 bondes con plazo 707 con cupón que paga 28 días (calcular cupón basado en curva de fondeo y descontar con la curva correspondiente). 

In [407]:
%%R

#CETES
base="RiesgosFinancieros/2022-1/Tarea/tasa_guber.txt"
ven_cete = cbind(180, 0)
contratos_cete = cbind(1500, 0)
nominal_cete = 10


#BONO M CON TASA FIJA
btasadesc="RiesgosFinancieros/2022-1/Tarea/tasa_yield.txt"
tf = cbind(0.045,0)
tfcupon = cbind(0.0405, 0)
ven_bonom = cbind(3600, 0)
plazocupon_bonom = cbind(182, 0)
contratos_bonom = cbind(1000, 0)
nominal_bonom = 100

#BONDES D
btasadescst="RiesgosFinancieros/2022-1/Tarea/tasa_guber_st.txt"
btasafondeo="RiesgosFinancieros/2022-1/Tarea/tfondeo.txt"
plazos_bdm= cbind(707, 707) #Vencimiento del bono
plazocupon_bdm= cbind(28, 28) #plazos_bdm fijos de cada cupón
contratos_bdm = cbind(1000, 0)
nominal_bdm=1000

###Inciso 4
100 contratos de compra de futuros de peso dólar con un strike de 20.83 vencimiento de 5 días, y 50 contratos
de venta de futuros del IPC con strike de 49525 vencimiento de 53 días.

In [408]:
%%R
#FORWARDS TDC
bext="RiesgosFinancieros/2022-1/Tarea/tasa_libor.txt"
bdom="RiesgosFinancieros/2022-1/Tarea/tasa_fwd.txt"
SymbolsFX_ftdc<-c("USDMXN=X", "GBPUSD=X" ) #tienen que ir en orden alfabético
plazos_fwd=cbind(5)
contratos_fwd=cbind(100)
kst_fwd=cbind(20.83)
nominal_fwd=1
yext=1 #si se carga información de yahoo en la fecha definida por fval o SymbolsFX, en caso contrario se utiliza información que se tendrá que cargar de tasas_spot.txt
trlib=1 #1 si la curva libor viene a 182 0 si no.

#FORWARDS DE IPC
#Descontamos con gubernamental
base="RiesgosFinancieros/2022-1/Tarea/tasa_guber.txt"
SymbolsEQ_find<-c("^MXX", "GCARSOA1.MX" ) #tienen que ir en orden alfabético
plazos_fwd_ind=cbind(53)
contratos_fwd_ind=cbind(50)
kst_fwd_ind=cbind(49525)
nominal_fwd_ind=1

### Inciso 5

Un swap largo nocional de 1600 pagando tasa fija de 4.6% anual (cada 28 días) y recibiendo tasa flotante de la curva TIIE vencimiento de 588 días, un swap corto con nocional de 1200 pagando tasa variable de la TIIE y recibiendo fija de 4.9% vencimiento de 270.

In [409]:
%%R
btasadesc_sw="RiesgosFinancieros/2022-1/Tarea/tasa_TIIE_SW_OP.txt"
btasacupvar_sw="RiesgosFinancieros/2022-1/Tarea/tasa_DIRS_SW_OP.txt"
tasafija_sw=cbind(0.046,0.049) #se establece la tasa fija a pagar para cada swap
plazos_sw=cbind(588,270) #se establece el número de días que vivirá el swap
plazocupon_sw=cbind(28,28) #se establece el número de días que se pagará cada cupón
contratos_sw=cbind(1600,-1200) #se establece el número de contratos_sw de cada swap
nominal_sw=cbind(1, 1) #se establece el nominal_sw de cada swap
por_sw=cbind(0,1) #se establece 0 si se paga tasa fija y 1 si se paga tasa variable

### Inciso 6
Dos opciones europeas, una de tasa de interés larga call con strike de 5.8 % vencimiento de 1700 días 1000
contratos, y otra put larga de tasa de interés con strike de 6.0 % de la tiie nocional de 500 contratos vencimiento
700 días. (se valuará basado en las superficies de volatilidad si es in o out the money y en las curvas de tasa de interés TIIE o Pagarés).

In [410]:
%%R
btasadesc_oir="RiesgosFinancieros/2022-1/Tarea/tasa_TIIE_SW_OP.txt" #Tasa de descuento
btasaspot_oir="RiesgosFinancieros/2022-1/Tarea/tasa_DIRS_SW_OP.txt" #Tasa Spot
bvolspot_oir="RiesgosFinancieros/2022-1/Tarea/tvoltiie_opc.txt" #Volatilidad
plazos_oir=cbind( 1700, 700) #T-t
pr_oir=28 #plazo de referencia  
dct_oir=360 #d_base
cp_oir=cbind(1,0) #si es call (cap) o put (floor) 
K_oir=cbind( 0.058,0.06) #Strike
contratos_oir=cbind(1000,500)
nominal_oir=1
cs_oir=1 #1 si es continua la tasa 0 si es simple

## **Carga de datos**

### Inciso 1

In [411]:
%%R
start_date=Sys.Date()-nh #fecha inicial
dataEnv<-new.env()
getSymbols.yahoo(Symbols,env=dataEnv,from=start_date)

bt.prep(dataEnv,align='remove.na',fill.gaps = T)
stock_prices = dataEnv$prices

###Inciso 2

In [412]:
%%R
  start_date=Sys.Date()-nh #fecha inicial
  dataEnvFX<-new.env() #Creamos donde guardar los datos
  getSymbols.yahoo(SymbolsFX,env=dataEnvFX,from=start_date) #Obtenemos los datps
  bt.prep(dataEnvFX,align='remove.na',fill.gaps=T) #Limpieza de datos
  stock_pricesFX = dataEnvFX$prices #Nos da el precio

  stock_pricesFX=cbind(stock_pricesFX[,1,with=F]*stock_pricesFX[,3,with=F],stock_pricesFX[,2,with=F]*stock_pricesFX[,3,with=F],stock_pricesFX[,3,with=F])
stock_prices_EQFX=merge(stock_pricesFX,stock_prices,join = "inner")

aux2=data.table(Date=as.Date(index(stock_prices_EQFX)),coredata(stock_prices_EQFX))


### Inciso 3

In [413]:
%%R

# CETES
data_cetes<-read.table(base)
n_gov<-nrow(data_cetes)
m_gov=ncol(data_cetes)
x_orig_gov=data.frame(data_cetes[2:n_gov,1:m_gov])
x_orig_gov=as.data.table(mutate(data_cetes[2:n_gov,1:m_gov],Date=as.Date(V1,format="%Y%m%d")))
x_orig_gov=x_orig_gov%>%select(-V1)
nodos_gov=data.frame(data_cetes[1,2:m_gov])

# BONO M
data_m<-read.table(btasadesc)
n_bonom<-nrow(data_m)
m_bonom=ncol(data_m)
x_orig_bonom=data.frame(data_m[2:n_bonom,1:m_bonom])
x_orig_bonom=as.data.table(mutate(data_m[2:n_bonom,1:m_bonom],Date=as.Date(V1,format="%Y%m%d")))
x_orig_bonom=x_orig_bonom%>%select(-V1)
nodos_bonom=data.frame(data_m[1,2:m_bonom])

# BONDE D

data1<-read.table(base)
n<-nrow(data1)
m_bd=ncol(data1)
X_orig_bd=as.data.table(mutate(data1[2:n,1:m_bd],Date=as.Date(V1,format="%Y%m%d")))
X_orig_bd%>%select(-V1)

X1_orig=mutate(data.frame(data1[2:n,1:m_bd]), V1=as.Date(V1,format="%Y%m%d"))
print(head(X1_orig[,1:4]))
nodos=data.frame(data1[1,2:m_bd])
n=n-1

data3<-read.table(btasadescst)
n3<-nrow(data3)
m3_bd=ncol(data3)
X3_orig_bd=as.data.table(mutate(data3[2:n,1:m3_bd],Date=as.Date(V1,format="%Y%m%d")))
X3_orig_bd=X3_orig_bd%>%select(-V1)

X3a_orig=mutate(data.frame(data1[2:n,1:m_bd]), V1=as.Date(V1,format="%Y%m%d"))
nodos3_bd=data.frame(data3[1,2:m3_bd])
n3=n3-1

data2<-read.table(btasafondeo)
n2<-nrow(data2)
X2_orig_bd=data.frame(data2[2:n2,1:2])
print(head(X2_orig_bd))
X2_orig_bd=mutate(X2_orig_bd, V1=as.Date(V1,format="%Y%m%d"), Date=as.Date(V1,format="%Y%m%d"), V2=as.numeric(as.character(V2)))
tfh=seq(min(X2_orig_bd$V1), max(X2_orig_bd$V1), "days") #sucesión de dias para tasa fondeo
tfhd=data.frame(ID=1:count(tfh),fecha=tfh)

#Cruzar la sucesión de todos los días versus el de tasa de fondeo
tfhd=setDT(tfhd)[, Date := tfh][order(-Date)]
X2_orig_bd=setDT(X2_orig_bd)[, Date := V1][order(-Date)]
# rolling join unión por rolling, rellena las fechas que faltaban con el último valor conocido "roll=Inf"
X2_orig_bd=X2_orig_bd[tfhd, on = .(Date), roll = Inf] 
print(head(X2_orig_bd))
#buscar fecha de valuación en tfondeo
tf_act=X2_orig_bd[fecha==fval,]$V2/100
tf_int=X2_orig_bd[fecha<=fval & fecha>=(fval-plazocupon_bdm[1])]$V2/100

X1_orig=setDT(X1_orig)[, Date:= V1][order(-Date)] #Para alinear con valor presente y tasa de fondeo.


          V1         V2         V3         V4
2 2020-03-06 0.07906000 0.07808887 0.07709264
3 2020-03-05 0.07722139 0.07808887 0.07709264
4 2020-03-04 0.07722139 0.07808887 0.07744709
5 2020-03-03 0.07813334 0.07808887 0.07744709
6 2020-03-02 0.07722139 0.07808887 0.07744709
7 2020-02-28 0.07722139 0.07802004 0.07744709
        V1    V2
2 19981103 30.75
3 19981104  29.2
4 19981105  29.8
5 19981106  31.3
6 19981109  32.9
7 19981110 34.54
           V1   V2       Date   ID      fecha
1: 2020-03-13 7.06 2020-03-13 7802 2020-03-13
2: 2020-03-12 7.10 2020-03-12 7801 2020-03-12
3: 2020-03-11 7.02 2020-03-11 7800 2020-03-11
4: 2020-03-10 6.98 2020-03-10 7799 2020-03-10
5: 2020-03-09 7.05 2020-03-09 7798 2020-03-09
6: 2020-03-06 7.04 2020-03-08 7797 2020-03-08


In [414]:
%%R
X2_orig_bd[fecha==fval,]$V2/100

[1] 0.0706


###Inciso 4


In [415]:
%%R
#CARGA DE DATOS DE  FORWARDS DE TDC
#datas
#data<-read.table("tasa_tiie.txt")
data1<-read.table(bext)
data2<-read.table(bdom)

########minimos para parametrizar
n1=nrow(data1)
n2=nrow(data2)
m1_ftdc=ncol(data1)
m2_ftdc=ncol(data2)
n=min(n1,n2)-1
###NODOS###
nodos1_ftdc=data.frame(data1[1,2:m1_ftdc])
nodos2_ftdc=data.frame(data2[1,2:m2_ftdc])
####MATRICES DEL MISMO TAMAÑO MENOS DOLAR

x1_ftdc=as.data.table(mutate(data1[2:n,1:m1_ftdc],Date=as.Date(V1,format="%Y%m%d")))

x1_ftdc=x1_ftdc%>%select(-V1)
x2_ftdc=as.data.table(mutate(data2[2:n,1:m2_ftdc],Date=as.Date(V1,format="%Y%m%d")))
x2_ftdc=x2_ftdc%>%select(-V1)
print(x2_ftdc[1:20,])

###Para Dolar

if (yext==1)
{
#Cargar los símbolos de yahoo finance para FX
  start_date=fval-3660 #fecha inicial


  
#Creación del objeto para guardar los datos
  dataEnvFX_ftdc<-new.env()
  
#obtener los datos
  getSymbols.yahoo(SymbolsFX_ftdc,env=dataEnvFX_ftdc,from=start_date, to=(fval))
#limpiarlos, alinearnos y quedarnos con el precio de cierre 
  bt.prep(dataEnvFX_ftdc,align='remove.na',fill.gaps=T)
  
#muestra de datos
  head(dataEnvFX_ftdc$prices[,2])
  
#Nos quedamos con los precios
  X3_ftdc=data.table(Date=as.Date(index(dataEnvFX_ftdc$prices[,2])),coredata(dataEnvFX_ftdc$prices[,2]))
} else  
{ 
data3<-read.table(btsp)
print(head(data3))
n3<-nrow(data3)
m3<-ncol(data3)
X3=data.table(as.matrix(as.double(as.matrix(data3[2:(n+1),m3]))))
X3_find=as.data.table(mutate(data3[2:(n+1),1:m3],Date=as.Date(V1,format="%Y%m%d")))
}


# CARGA DE DATOS DE  FORWARD DE IPC

data<-read.table(base)
n<-nrow(data)
m_gov=ncol(data)
#x_orig_gov=data.frame(data[2:n,1:m_gov])
x_orig_gov=as.data.table(mutate(data[2:n,1:m_gov],Date=as.Date(V1,format="%Y%m%d")))
x_orig_gov=x_orig_gov%>%select(-V1)
nodos_gov=data.frame(data[1,2:m_gov])

  #Cargar los símbolos de yahoo finance para EQ
  start_date=fval-nh #fecha inicial
  
  #Creación del objeto para guardar los datos
  dataEnvEQ<-new.env()
  
  #obtener los datos
  getSymbols.yahoo(SymbolsEQ_find,env=dataEnvEQ,from=start_date, to=(fval))
  #limpiarlos, alinearnos y quedarnos con el precio de cierre 
  bt.prep(dataEnvEQ,align='remove.na',fill.gaps=T)
  
  #muestra de datos
#  head(dataEnvEQ$prices)
  
  #Nos quedamos con los precios
  X3_find=data.table(Date=as.Date(index(dataEnvEQ$prices[,2])),coredata(dataEnvEQ$prices[,2]))

          V2       V3       V4       V5       V6       V7       V8       V9
 1: 7.818577 7.818577 7.839789 8.067963 8.239547 8.343793 8.472818 8.635362
 2: 7.784639 7.805316 7.838284 8.068495 8.247257 8.343650 8.472873 8.616668
 3: 7.817477 7.829912 7.837796 8.076634 8.247080 8.343621 8.472850 8.616802
 4: 7.770354 7.795126 7.840694 8.067389 8.231023 8.342564 8.463999 8.616986
 5: 7.799087 7.816851 7.834036 8.059198 8.230909 8.333645 8.455073 8.598360
 6: 7.781644 7.792078 7.834036 8.059198 8.230909 8.333645 8.455073 8.598360
 7: 7.782668 7.804714 7.834036 8.059198 8.230909 8.333645 8.455073 8.598505
 8: 7.799087 7.816851 7.834036 8.059198 8.233134 8.350175 8.464103 8.617028
 9: 7.799087 7.816851 7.834036 8.059198 8.233134 8.350175 8.464053 8.626142
10: 7.766249 7.792575 7.834036 8.059428 8.243066 8.367934 8.481755 8.691225
11: 7.810369 7.810372 7.831645 8.060029 8.243071 8.366564 8.472838 8.681864
12: 7.807297 7.819819 7.827540 8.058986 8.223236 8.341373 8.455107 8.616715
13: 7.831926

### Inciso 5

In [416]:
%%R
data1<-read.table(btasadesc_sw)
n1<-nrow(data1)
m1_orig_sw=ncol(data1)
X1_orig_sw=data.table(mutate(data1[2:n1,1:m1_orig_sw],Date=as.Date(V1,format="%Y%m%d")))
X1_orig_sw=X1_orig_sw%>%select(-V1)
nodos1_sw=data.frame(data1[1,2:m1_orig_sw])
  
data2<-read.table(btasacupvar_sw)
n2<-nrow(data2)
m2_orig_sw=ncol(data2)
X2_orig_sw=data.table(mutate(data2[2:n1,1:m1_orig_sw],Date=as.Date(V1,format="%Y%m%d")))
X2_orig_sw=X2_orig_sw%>%select(-V1)
nodos2_sw=data.frame(data2[1,2:m2_orig_sw])


###Inciso 6

In [417]:
%%R
  #carga de rho	
  data1<-read.table(btasadesc_oir) 
  n<-nrow(data1)
  m1_orig_oir=ncol(data1)
  x1_orig_oir=data.table(mutate(data1[2:n,1:m1_orig_oir],Date=as.Date(V1,format="%Y%m%d")))
  x1_orig_oir=x1_orig_oir%>%select(-V1)
  nodos1_oir=data.frame(data1[1,2:m1_orig_oir])
  
  #data1[1:3,]
  #carga de tasas spot
  data2<-read.table(btasaspot_oir)
  n<-nrow(data2)
  m2_orig_oir=ncol(data2)
  x2_orig_oir=data.table(mutate(data2[2:n,1:m2_orig_oir],Date=as.Date(V1,format="%Y%m%d")))
  x2_orig_oir=x2_orig_oir%>%select(-V1)
  nodos2_oir=data.frame(data2[1,2:m2_orig_oir])
  #carga de volatilidades de spot
  data3<-read.table(bvolspot_oir)
  n<-nrow(data3)
  m3_orig_oir=ncol(data3)
  x3_orig_oir=data.table(mutate(data3[2:n,1:m3_orig_oir],Date=as.Date(V1,format="%Y%m%d")))
  x3_orig_oir=x3_orig_oir%>%select(-V1)
  
  nodos3_oir=data.frame(data3[1,2:m3_orig_oir])

##**Integración de Insumos**

### Cruce de tablas

In [418]:
%%R
#INTERSECCIÓN DE FECHAS DE TODOS LOS INSUMOS    
  

lin_gub=data.table(Date=as.Date(aux2[x_orig_gov,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, yield

#lin_gub_aux=data.table(Date=as.Date(lin_gub[x_orig_bonom,on=.(Date),nomatch=0]$Date)) 

lin_gub_bmybdst_aux=data.table(Date=as.Date(lin_gub[X3_orig_bd,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, yield y st (bonde)


lin_gub_bmybdst=data.table(Date=as.Date(X3_ftdc[lin_gub_bmybdst_aux,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, yield y st (bonde)

lin_gub_bmybdst_flib=data.table(Date=as.Date(lin_gub_bmybdst[x1_ftdc,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, yield,  st (bonde), libor

lin_gub_bmybdst_flibfwd=data.table(Date=as.Date(lin_gub_bmybdst_flib[x2_ftdc,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, yield, st (bonde), libor, fwd

lin_gub_bmybdst_flibfwdspind=data.table(Date=as.Date(lin_gub_bmybdst_flibfwd[X3_find,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, yield, st (bonde), libor, fwd, spot, equity or index



lin_gub_bmybdst_flibfwdspind_swcup=data.table(Date=as.Date(lin_gub_bmybdst_flibfwdspind[X1_orig_sw,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, yield, st (bonde), libor, fwd, spot, equity or index, swap cupón

lin_gub_bmybdst_flibfwdspind_swcupvp=data.table(Date=as.Date(lin_gub_bmybdst_flibfwdspind_swcup[X2_orig_sw,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, yield, st (bonde), libor, fwd, spot, equity or index, swap cupón

lin_gub_bmybdst_flibfwdspind_swcupvp_oirs=data.table(Date=as.Date(lin_gub_bmybdst_flibfwdspind_swcupvp[x1_orig_oir,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, yield, st (bonde), libor, fwd, spot, equity or index, swap cupón, opciones ir spot

lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvp=data.table(Date=as.Date(lin_gub_bmybdst_flibfwdspind_swcupvp_oirs[x2_orig_oir,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, yield, st (bonde), libor, fwd, spot, equity or index, swap cupón, opciones ir spot, opc ir vp

lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol=data.table(Date=as.Date(lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvp[x3_orig_oir,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, yield, st (bonde), libor, fwd, spot, equity or index, swap cupón, opciones ir spot, opc ir vp, opc ir vol


lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol=unique(lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol)
print(lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol)

           Date
  1: 2020-03-06
  2: 2020-03-05
  3: 2020-03-04
  4: 2020-03-03
  5: 2020-03-02
 ---           
216: 2019-03-26
217: 2019-03-25
218: 2019-03-21
219: 2019-03-20
220: 2019-03-19


### Formato de historia

#### Historia de todos

In [419]:
%%R
n=nrow(lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol)
n

[1] 220


#### Incisos 1 y 2

In [420]:
%%R
#historia de acciones y divisas
stock_prices_EQFX=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[aux2,on=.(Date),nomatch=0][order(-Date)]
stock_prices_EQFX=stock_prices_EQFX%>%select(-Date)

#### Inciso 3

In [421]:
%%R
#Historia de curva gubernamental y yield
x_orig_gov=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x_orig_gov,on=.(Date),nomatch=0][order(-Date)]
x_orig_gov=x_orig_gov%>%select(-Date)

#Historia de curvas de bonde
X1_orig=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[X1_orig,on=.(Date),nomatch=0][order(-Date)]
X1_orig=X1_orig%>%select(-Date)
X2_orig_bd=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[X2_orig_bd,on=.(Date),nomatch=0][order(-Date)]
X2_orig_bd=X2_orig_bd%>%select(-Date)
X3_orig_bd=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[X3_orig_bd,on=.(Date),nomatch=0][order(-Date)]
X3_orig_bd=X3_orig_bd%>%select(-Date)



#### Inciso 4


In [422]:
%%R
#historia de curvas de forward tdc
x1_ftdc=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x1_ftdc,on=.(Date),nomatch=0][order(-Date)]
x1_ftdc=x1_ftdc%>%select(-Date)/100
x2_ftdc=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x2_ftdc,on=.(Date),nomatch=0][order(-Date)]
x2_ftdc=x2_ftdc%>%select(-Date)/100
X3_ftdc=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[X3_ftdc,on=.(Date),nomatch=0][order(-Date)]
X3_ftdc=X3_ftdc%>%select(-Date)

#historia de curvas de forward ind
#CONSIDERAR LA CURVA GUBERNAMENTAL X1_ORIG_GOV
X3_find=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[X3_find,on=.(Date),nomatch=0][order(-Date)]
X3_find=X3_find%>%select(-Date)

#### Inciso 5

In [423]:
%%R
X1_orig_sw=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[X1_orig_sw,on=.(Date),nomatch=0][order(-Date)]
X1_orig_sw=X1_orig_sw%>%select(-Date)/100
X2_orig_sw=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[X2_orig_sw,on=.(Date),nomatch=0][order(-Date)]
X2_orig_sw=X2_orig_sw%>%select(-Date)/100

####Inciso 6

In [424]:
%%R
x1_orig_oir=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x1_orig_oir,on=.(Date),nomatch=0][order(-Date)]
x1_orig_oir=x1_orig_oir%>%select(-Date)/100
x2_orig_oir=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x2_orig_oir,on=.(Date),nomatch=0][order(-Date)]
x2_orig_oir=x2_orig_oir%>%select(-Date)/100
x3_orig_oir=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x3_orig_oir,on=.(Date),nomatch=0][order(-Date)]
x3_orig_oir=x3_orig_oir%>%select(-Date)

## **Valuación**

### Inciso 1 y 2: Acciones y divisas

In [425]:
%%R
x0_acc_div=stock_prices_EQFX[1,]
DeltaX_acc_div=as.matrix(log(as.matrix(stock_prices_EQFX[1:(n-1)])/as.matrix(stock_prices_EQFX[2:(n)])))
V0_acc_div=cbind(t(pos_fx),t(pos_eq))*x0_acc_div
m_fx=length(pos_fx)
m_acc=length(pos_eq)

### Inciso 3: Cetes

In [426]:
%%R
# Interpolamos las tasas
m=ncol(ven_cete)
x=matrix(0,n,m)
for (i in 1:(n))
{
    if(itpl == 0){
        x[i,]=approx(nodos_gov,x_orig_gov[i,], ven_cete)$y
    }else{
        x[i,]=talamb(nodos_gov,x_orig_gov[i,],ven_cete)
    }
}
#Función para el calculo de los bonos cupon 0
bonocupcero = function(i,t)
{
1/(1+i*t/360)
}

#Valor actual de cada bono
x0_cetes=x[1,]
V0_cetes=as.matrix(bonocupcero(x0_cetes,ven_cete))*nominal_cete 
V0_cetes[1,2] = 0
#Valor total
VT0_cetes=sum(V0_cetes)

VT0_cetes


[1] 9.615927


In [427]:
%%R
nrow(X_futind)



[1] 220


### Inciso 3: Bonos M

In [428]:
%%R
#interpolación de tasas y volatilidades
#revisar la valuacion que cambia si usamos n en vez de n-1
m=ncol(ven_bonom)
X=matrix(0,n,m)
for (i in 1:(n))
{
    if(itpl == 0){
        X[i,]=approx(nodos_bonom,x_orig_gov[i,], m)$y
    }else{
        X[i,]=talamb(nodos_bonom,x_orig_gov[i,],m)
    }
}

bonoMyield=function(x, plazos, plazocupon, tfcupon, nominal, contratos = 1) #valoración bono tasa fija
{
x=x0  
N=as.integer(plazos/plazocupon)+1   
a=(1-(1+x*plazocupon/360)^(-N))/(plazocupon*x/360)
p1=plazos-plazocupon*(N-1)   
((contratos*nominal*tfcupon*plazocupon/360)*a+(contratos*nominal)/((1+x*plazocupon/360)^N))*(1+x*plazocupon/360)^(1-p1/plazocupon)   
}

x0=X[1,] #tasas de descuento valor actual
V0_bonom = bonoMyield(x0,ven_bonom, plazocupon_bonom, tfcupon, nominal_bonom, contratos_bonom)
V0_bonom[1,2] = 0
VT0_bonom=sum(V0_bonom)
VT0_bonom


[1] 74226.93


In [429]:
%%R

X2_orig_bd

             V1   V2   ID      fecha
  1: 2020-03-06 7.04 7795 2020-03-06
  2: 2020-03-05 7.06 7794 2020-03-05
  3: 2020-03-04 7.05 7793 2020-03-04
  4: 2020-03-03 7.07 7792 2020-03-03
  5: 2020-03-02 7.05 7791 2020-03-02
 ---                                
216: 2019-03-26 8.27 7449 2019-03-26
217: 2019-03-25 8.28 7448 2019-03-25
218: 2019-03-21 8.27 7444 2019-03-21
219: 2019-03-20 8.27 7443 2019-03-20
220: 2019-03-19 8.26 7442 2019-03-19


### Inciso 3: Bondes D

In [430]:
%%R
#BONDE D CÁLCULO
names(X2_orig_bd)[1] = 'Date'
X2_pr=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[X2_orig_bd, on = .(Date),nomatch=0][order(-Date)] #Se alinea la tasa de fondeo por fecha con la tasa de valor presente.
m=ncol(plazos_bdm)


N_bd=as.integer(plazos_bdm/plazocupon_bdm)+1 #número de cupones a pagar
VTplazos_bdm=matrix(0,1,sum(N_bd)) #vector de todos los plazos_bdm de todos los contratos_bdm
contratos_bdmT=matrix(0,1,sum(N_bd)) #vector de todos los contratos_bdm de todos los flujos de todos los contratos_bdm
nominal_bdmT=matrix(0,1,sum(N_bd)) #vector de todos los nominal_bdmes de todos los flujos de todos los contratos_bdm
plazocupon_bdmT=matrix(0,1,sum(N_bd)) #vector de todos los plazos_bdmcupon de todos los flujos de todos los contratos_bdm
tasafijaT_bd=matrix(0,1,sum(N_bd)) #vector de tasas fijas de todos los flujos de todos los contratos_bdm
ulNomT_bd=matrix(0,1,sum(N_bd)) #vector de contratos_bdm a final de flujo


plazini_bd=plazos_bdm-plazocupon_bdm*(N_bd-1) #vector de plazos_bdm iniciales
ddv=plazocupon_bdm-plazini_bd #dias trasncurridos del cupón vigente
tfcupon=matrix(0,1,m)  #El primero cupón de cada bono
tfcupondev=matrix(0,1,m) #cupón de los días devengados
tfcupgen=((1+tf_act/360)^(plazocupon_bdm[1])-1)*360/plazocupon_bdm[1] #el segundo al último cupón de todos los bonos
#calcula cupones de bonos
for (j in (1:m))
{
  tfcupondev[j]=(prod(1+tf_int[(1:ddv[j])]/360)-1)*360/ddv[j]
  tfcupon[j]=((1+tfcupondev[j]*ddv[j]/360)*(1+tf_act/360)^(plazocupon_bdm[1]-ddv[j])-1)*360/plazocupon_bdm[1]
}


for (j in (1:m))
{
  if (j==1)
  {
    VTplazos_bdm[,1:sum(N_bd[1:j])]=seq(plazini_bd[j],plazos_bdm[j], by=plazocupon_bdm[j])
    contratos_bdmT[,1:sum(N_bd[1:j])]=seq(contratos_bdm[j],contratos_bdm[j])
    plazocupon_bdmT[,1:sum(N_bd[1:j])]=seq(plazocupon_bdm[j],plazocupon_bdm[j])
    ulNomT_bd[,sum(N_bd[1:j])]=contratos_bdm[j]
    tasafijaT_bd[,1]=tfcupon[j]
    tasafijaT_bd[,2:sum(N_bd[1:j])]=seq(tfcupgen,tfcupgen)
  }
  else
  {
    VTplazos_bdm[,(sum(N_bd[1:j-1])+1):sum(N_bd[1:j])]=seq(plazini_bd[j],plazos_bdm[j], by=plazocupon_bdm[j])
    contratos_bdmT[,(sum(N_bd[1:j-1])+1):sum(N_bd[1:j])]=seq(contratos_bdm[j],contratos_bdm[j])					
    plazocupon_bdmT[,(sum(N_bd[1:j-1])+1):sum(N_bd[1:j])]=seq(plazocupon_bdm[j],plazocupon_bdm[j])
    tasafijaT_bd[,(sum(N_bd[1:j-1])+1)]=tfcupon[j]
    tasafijaT_bd[,(sum(N_bd[1:j-1])+2):sum(N_bd[1:j])]=seq(tfcupgen,tfcupgen)
    ulNomT_bd[,sum(N_bd[1:j])]=contratos_bdm[j]
  }
}

Xvp_bd=matrix(0,n,ncol(VTplazos_bdm))
Xst_bd=matrix(0,n,ncol(VTplazos_bdm))

for (i in (1:n))
{
  Xvp_bd[i,]=if(itpl==0){approx(nodos_gov,x_orig_gov[i,],VTplazos_bdm,rule=2)$y}else{talamb(nodos_gov,x_orig_gov[i,],VTplazos_bdm)}
  Xst_bd[i,]=if(itpl==0){approx(nodos3_bd,X3_orig_bd[i,],VTplazos_bdm,rule=2)$y}else{talamb(nodos3_bd,X3_orig_bd[i,],VTplazos_bdm)}
  
}

X_bd_tc=matrix(1,n,ncol(contratos_bdmT))*X2_pr$V2/100
X_bd_ext=cbind(X_bd_tc,as.matrix(Xvp_bd),as.matrix(Xst_bd))

bondeD=function(contratos_bdmT,  nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp, Xst, N,ddv)
{
  tfcupon=matrix(0,1,m)  #El primero cupón de cada bono
  tfcupondev=matrix(0,1,m) #cupón de los días devengados
  tfcupgen=((1+tf_act/360)^(plazocupon_bdm[1])-1)*360/plazocupon_bdm[1] #el segundo al último cupón de todos los bonos
  tasafijaT=matrix(0,1,sum(N))
  #calcula cupones de bonos
  for (j in (1:m))
  {
    tfcupondev[j]=(prod(1+tf_int[(1:ddv[j])]/360)-1)*360/ddv[j]
    tfcupon[j]=((1+tfcupondev[j]*ddv[j]/360)*(1+tf_act/360)^(plazocupon_bdm[1]-ddv[j])-1)*360/plazocupon_bdm[1]
  }
  
  
  for (j in (1:m))
  {
    if (j==1)
    {
      tasafijaT[,1]=tfcupon[j]
      tasafijaT[,2:sum(N[1:j])]=seq(tfcupgen,tfcupgen)
    }
    else
    {
      tasafijaT[,(sum(N[1:j-1])+1)]=tfcupon[j]
      tasafijaT[,(sum(N[1:j-1])+2):sum(N[1:j])]=seq(tfcupgen,tfcupgen)
    }
  }
  
  
  V0=matrix(0,1,count(N))
  V0f=((((contratos_bdmT*(tasafijaT)*(plazocupon_bdmT/360))+ulNomT_bd)/(1+(Xvp+Xst)*VTplazos_bdm/360)))*nominal_bdm
  for (j in (1:count(N)))
  {
    if(j==1)
    {
      V0[j]=sum(V0f[j:N[j]])
    }
    else
    {
      V0[j]=sum(V0f[(sum(N[1:j-1])+1):(sum(N[1:j]))])
    }
  }
  V0
} 



V0_bd=bondeD(contratos_bdmT, nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp_bd[1,], Xst_bd[1,], N_bd,ddv)

V0_bd

         [,1] [,2]
[1,] 951537.5    0


### Inciso 4: Forwards de tipo de cambio

In [431]:
%%R
#FORWARDS Y/O FUTUROS DE TIPO DE CAMBIO CÁLCULO

################MATRICES DE INTERPOLACION LINEAL ####################

m=ncol(plazos_fwd)
X1_fwtdc=matrix(0,n,m)
X2_fwtdc=matrix(0,n,m)


for (j in 1:n)
{
  X1_fwtdc[j,]=if(itpl==0){approx(nodos1_ftdc,x1_ftdc[j,],plazos_fwd,rule=2)$y}else{talamb(nodos1_ftdc,x1_ftdc[j,],plazos_fwd)}
  X2_fwtdc[j,]=if(itpl==0){approx(nodos2_ftdc,x2_ftdc[j,],plazos_fwd,rule=2)$y}else{talamb(nodos2_ftdc,x2_ftdc[j,],plazos_fwd)}
  if(trlib==1){X1_fwtdc[j,]=((1+X1_fwtdc[j,])^(plazos_fwd/180)-1)*360/plazos_fwd} #transformación de act 180 a 360
}


futuroTC = function(t,tl,tn,s,k) #t=dias por vencer, tn=tasa nacional para tipo de cambio forward, tl= tasa extranjera pra tipo de cambio forward, S=spot
{
  f=s*((1+tn*t/360)/(1+tl*t/360)) #Se obtiene el tipo de cambio forward
  t(as.numeric((f-k)/(1+t*tn/360))) #Se obtiene el valor del payoff a valor presente con el valor z que define si es largo o corto
}

X3_ftdc=as.matrix(X3_ftdc)
X_futtdc=cbind(X1_fwtdc,X2_fwtdc,X3_ftdc)

V0_fwtdc=futuroTC(plazos_fwd,X1_fwtdc[1,],X2_fwtdc[1,],X3_ftdc[1,],kst_fwd)*contratos_fwd*nominal_fwd
# V0_fwtdc=futuroTC(plazos_fwd,X1_fwtdc[1,],X2_fwtdc[1,],X3_ftdc[1,],kst_fwd)*contratos_fwd*nominal_fwd]
V0_fwtdc

          [,1]
[1,] -100.3609


### Inciso 4: Forwards de indice

In [432]:
%%R
#FORWARDS Y/O FUTUROS DE ÍNDICES CÁLCULO

################MATRICES DE INTERPOLACION LINEAL ####################

m_ind=ncol(plazos_fwd_ind)
X1_fwind=matrix(0,n,m_ind) #DIVIDENDOS
X2_fwind=matrix(0,n,m_ind)

for (j in 1:n)
{
  #X1_fwind[j,]=if(itpl==0){approx(nodos1_,x1_ftdc[j,],plazos_fwd)$y}else{talamb(nodos1_ftdc,x1_ftdc[j,],plazos_fwd)}
  X2_fwind[j,]=if(itpl==0){approx(nodos_gov,x_orig_gov[j,],plazos_fwd_ind,rule=2)$y}else{talamb(nodos_gov,x_orig_gov[j,],plazos_fwd_ind)}
  #if(trlib==1){X1_fwtdc[j,]=((1+X1_fwtdc[j,])^(plazos_fwd/180)-1)*360/plazos_fwd} #transformación de act 180 a 360
}

X3_find=as.matrix(X3_find)
X_futind=cbind(X1_fwind,X2_fwind,matrix(X3_find,n,ncol(X1_fwind)))

V0_fwind=futuroTC(plazos_fwd_ind,X1_fwind[1,],X2_fwind[1,],X3_find[1,],kst_fwd_ind)*contratos_fwd_ind*nominal_fwd_ind
V0_fwind


        [,1]
[1,] -378765


### Inciso 5: Swaps

In [433]:
%%R
nodosvp=nodos1_sw
nodostc=nodos2_sw
curvavp=as.matrix(X1_orig_sw)
curvatc=X2_orig_sw
n1=nrow(curvavp)
n2=nrow(curvatc)


m=max(ncol(plazos_sw),1) #número de contratos_sw swap a valorar
N=matrix(0,1,m) #es un vector de m valores donde se cargarán los m número de cupones a pagar para cada contrato
for (j in (1:m))
{
  N[j]=as.integer(plazos_sw[j]/plazocupon_sw[j])+1 #número de cupones a pagar
}
VTplazos_sw=matrix(0,1,sum(N)) #vector de todos los plazos_sw de todos los contratos_sw
contratos_swT=matrix(0,1,sum(N)) #vector de todos los contratos_sw de todos los flujos de todos los contratos_sw
nominal_swT=matrix(0,1,sum(N)) #vector de todos los nominal_swes de todos los flujos de todos los contratos_sw
por_swT=matrix(0,1,sum(N)) #vector de todos los dummy si paga o recibe de todos los flujos de todos los contratos_sw
plazocupon_swT=matrix(0,1,sum(N)) #vector de todos los plazos_swcupon de todos los flujos de todos los contratos_sw
tasafija_swT=matrix(0,1,sum(N)) #vector de tasas fijas de todos los flujos de todos los contratos_sw
VTplazos_swc=matrix(0,1,sum(N)) #vector de todos los plazos_sw cortos de todos los contratos_sw


plazini=plazos_sw-plazocupon_sw*(N-1) #vector de plazos_sw iniciales

for (j in (1:m))
{
  if (j==1)
  {
    VTplazos_sw[,1:sum(N[1:j])]=seq(plazini[j],plazos_sw[j], by=plazocupon_sw[j])
    VTplazos_swc[,1:sum(N[1:j])]=c(0,VTplazos_sw[,1:(sum(N[1:j])-1)])
    contratos_swT[,1:sum(N[1:j])]=seq(contratos_sw[j],contratos_sw[j])
    nominal_swT[,1:sum(N[1:j])]=seq(nominal_sw[j],nominal_sw[j])
    por_swT[,1:sum(N[1:j])]=seq(por_sw[j],por_sw[j])
    plazocupon_swT[,1:sum(N[1:j])]=seq(plazocupon_sw[j],plazocupon_sw[j])
    tasafija_swT[,1:sum(N[1:j])]=seq(tasafija_sw[j],tasafija_sw[j])
  }
  else
  {
    VTplazos_sw[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(plazini[j],plazos_sw[j], by=plazocupon_sw[j])
    VTplazos_swc[,(sum(N[1:j-1])+1):sum(N[1:j])]=c(0,VTplazos_sw[,(sum(N[1:j-1])+1):(sum(N[1:j])-1)])
    contratos_swT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(contratos_sw[j],contratos_sw[j])					
    nominal_swT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(nominal_sw[j],nominal_sw[j])
    por_swT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(por_sw[j],por_sw[j])
    plazocupon_swT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(plazocupon_sw[j],plazocupon_sw[j])
    tasafija_swT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(tasafija_sw[j],tasafija_sw[j])
  }
}


Xvp=matrix(0,n,ncol(VTplazos_sw))
Xtc=matrix(0,n,ncol(VTplazos_sw))
Xtcc=matrix(0,n,ncol(VTplazos_sw))
XtfwdT=matrix(0,n,ncol(VTplazos_sw))

for (i in (1:n))
{
  Xvp[i,]=if(itpl==0){approx(nodosvp,curvavp[i,],VTplazos_sw,rule=2)$y}else{talamb(nodosvp,curvavp[i,],VTplazos_sw)}
  Xtc[i,]=if(itpl==0){approx(nodostc,curvatc[i,],VTplazos_sw,rule=2)$y}else{talamb(nodostc,curvatc[i,],VTplazos_sw)}
  Xtcc[i,]=if(itpl==0){approx(nodostc,curvatc[i,],VTplazos_swc, rule=2)$y}else{talamb(nodostc,curvatc[i,],VTplazos_swc)}
  
  XtfwdT[i,]=((1+Xtc[i,]*VTplazos_sw/360)/(1+Xtcc[i,]*VTplazos_swc/360)-1)*360/plazocupon_swT 
  for (j in (1:ncol(VTplazos_sw)))
  {
    if (VTplazos_sw[j]<= plazocupon_swT[j])
    {
      XtfwdT[i,j]=Xtc[i,j]
    }
    else
    {
      j=sum(N[1:j])
    }
  }
}

X_sw=cbind(XtfwdT,Xvp)

swap=function(por_swT, contratos_swT, nominal_swT, XtfwdT, tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp, N)
{
  V0=matrix(0,1,ncol(N))
  V0f=(((contratos_swT*(XtfwdT-tasafija_swT)*(plazocupon_swT/360)))/(1+Xvp*VTplazos_sw/360))*nominal_swT*(-1)^por_swT
  for (j in (1:ncol(N)))
  {
    if(j==1)
    {
      V0[j]=sum(V0f[j:N[j]])
    }
    else
    {
      V0[j]=sum(V0f[(sum(N[1:j-1])+1):(sum(N[1:j]))])
    }
  }
  V0
}

V0_sw=swap(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp[1,], N)
V0_sw

         [,1]     [,2]
[1,] 79.69671 28.35638


###Inciso 6: Opciones

In [434]:
%%R
m=ncol(plazos_oir)
x1=matrix(0,n,m)
x2tc=matrix(0,n,m)
x2tl=matrix(0,n,m)
x2=matrix(0,n,m)
x3=matrix(0,n,m)
for (i in 1:(n))
{
  x1[i,]=if(itpl==0){approx(nodos1_oir,x1_orig_oir[i,],plazos_oir,rule=2)$y}else{talamb(nodos1_oir,x1_orig_oir[i,],plazos_oir)}
  x2tc[i,]=if(itpl==0){approx(nodos2_oir,x2_orig_oir[i,],plazos_oir,rule=2)$y}else{talamb(nodos2_oir,x2_orig_oir[i,],plazos_oir)}
  x2tl[i,]=if(itpl==0){approx(nodos2_oir,x2_orig_oir[i,],(plazos_oir+pr_oir),rule=2)$y}else{talamb(nodos2_oir,x2_orig_oir[i,],(plazos_oir+pr_oir))}
  x3[i,]=if(itpl==0){approx(nodos3_oir,x3_orig_oir[i,],plazos_oir,rule=2)$y}else{talamb(nodos3_oir,x3_orig_oir[i,],plazos_oir)}
  x2[i,]=((1+x2tl[i,]*(plazos_oir+pr_oir)/360)/(1+x2tc[i,]*(plazos_oir)/360)-1)*360/pr_oir
}

x01=x1[1,] #tasas de descuento
x02=x2[1,] #tasas spot
x03=x3[1,] #volatilidades

X_oir=cbind(x1,x2,x3)

opctint = function(d,S,K_oir,vol,t,cp_oir,cs_oir,pr_oir,dct_oir)	#función de una opción europea
{
  d1=if(cs_oir==1){(log(S/K_oir)+vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)+vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
  d2=if(cs_oir==1){(log(S/K_oir)-vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)-vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
  vp=if(cs_oir==1){log(1+d*t/360)*365/t}else{d}  
  (if(cs_oir==1){(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(exp(-vp*t/365))*(-1)^cp_oir}else{(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(1/(1+vp*t/360))*(-1)^cp_oir})*(pr_oir/dct_oir)/(1+S*pr_oir/dct_oir)
}

#Valor del portafolio
V0_oir=opctint(x01,x02,K_oir,x03,plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)*contratos_oir*nominal_oir
V0_oir

          [,1]      [,2]
[1,] 0.7591073 0.4324731


## Integración de factores de riesgo

In [442]:
%%R
n_if

     [,1]
[1,]    6
[2,]  156
[3,]   64
[4,]    6
[5,]    3
[6,]    3
[7,]    1
[8,]    1


In [436]:
%%R
#DIMENSION DE TODOS LOS INSTRUMENTOS
#Son 8 instrumentos financieros (9 si separamos acciones y divisas)
n_if=matrix(0,8,1)
n_if[1]=ncol(stock_prices_EQFX) #acciones y divisas
n_if[2]=ncol(X_bd_ext) #bonde
n_if[3]=ncol(X_sw) #swaps
n_if[4]=ncol(X_oir) #opciones tasa de interés
n_if[5]=ncol(X_futtdc) #Forwards de tipo de cambio
n_if[6]=ncol(X_futind) #Forwards de índices
n_if[7]=1 #Cetes
n_if[8]=1 #Bonos M
#valor del portafolios

V0_port=cbind(V0_acc_div, V0_bd, V0_sw, V0_oir,V0_fwtdc,V0_fwind, V0_cetes, VT0_bonom)
V0T_port=sum(V0_port)



#INTEGRACIÓN DE TODOS LOS FACTORES DE RIESGO EN UNA MATRIZ
X_port=cbind(stock_prices_EQFX,X_bd_ext,X_sw,X_oir,X_futtdc,X_futind, x[,1], X[,1]) #Factores de riesgo del portafolios de 8(9) instrumentos financieros

#Cálculo de variaciones Delta_X DEL PORTAFOLIOS
DeltaX_port=as.matrix(log(X_port[1:(n-1)]/X_port[2:(n)]))
DeltaX_port[is.nan(DeltaX_port)] <- 0 #quitamos NaN
DeltaX_port[is.na(DeltaX_port)] <- 0 #quitamos Na
DeltaX_port[is.infinite(DeltaX_port)] <- 0 #quitamos Na


Ns=nrow(DeltaX_port) #Definimos número de escenarios históricos

DeltaX_s=DeltaX_port

In [445]:
%%R
nrow(DeltaX_s)

[1] 219


## Medicion de riesgo, sin alisado

### Inciso 1 y 2: Acciones y divisas

In [437]:
%%R
m=m_fx+m_acc    #PASO CLAVE
X_s_acc_div=matrix(0,Ns,n_if[1]) #Factores de riesgo simulados con base en DeltaX_s x0*(1+Delta_Xs)   #PASO CLAVE
V_acc_div=matrix(0,Ns,m)
Vfr1_acc_div=matrix(0,Ns,m_fx)
Vfr2_acc_div=matrix(0,Ns,m_acc)
PG_acc_div=matrix(0,Ns,m) #Pèrdidas y ganancias
PGfr1_acc_div=matrix(0,Ns,m_fx)
PGfr2_acc_div=matrix(0,Ns,m_acc)
PGT_acc_div=matrix(0,Ns,1)
PGfr1T_acc_div=matrix(0,Ns,1)
PGfr2T_acc_div=matrix(0,Ns,1)

DeltaX_s_acc_div=DeltaX_s[,(1:n_if[1])]    #PASO CLAVE
x0_acc_div=stock_prices_EQFX[1,]   #PASO CLAVE

for (i in 1:Ns)
{
  X_s_acc_div[i,]=as.matrix(x0_acc_div*exp(DeltaX_s_acc_div[i,]))
  #PASO CLAVE
  V_acc_div[i,]=cbind(t(pos_fx),t(pos_eq))*X_s_acc_div[i,]
  #PASO CLAVE
  Vfr1_acc_div[i,]=t(pos_fx)*X_s_acc_div[i,1:m_fx]
  #PASO CLAVE
  Vfr2_acc_div[i,]=t(pos_eq)*X_s_acc_div[i,(m_fx+1):(m_fx+m_acc)]
  #PASO CLAVE
  PG_acc_div[i,]=as.matrix(V_acc_div[i,]-V0_acc_div)
  PGfr1_acc_div[i,]=as.matrix(Vfr1_acc_div[i,]-V0_acc_div[,1:m_fx])
  PGfr2_acc_div[i,]=as.matrix(Vfr2_acc_div[i,]-V0_acc_div[,(m_fx+1):(m_fx+m_acc)])
  PGT_acc_div[i,]=sum(PG_acc_div[i,])
  PGfr1T_acc_div[i,]=sum(PGfr1_acc_div[i,])
  PGfr2T_acc_div[i,]=sum(PGfr2_acc_div[i,])
}



PG_acc_div[1:5,]
PGfr1_acc_div[1:5,]
PGfr2_acc_div[1:5,]
PGT_acc_div[1:5,]


#VaR por posición
VaRCont_acc_div=matrix(0,1,m)
VaRfr1_acc_div=matrix(0,1,m_fx)
VaRfr2_acc_div=matrix(0,1,m_acc)
CVaRCont_acc_div=matrix(0,1,m)
CVaRfr1_acc_div=matrix(0,1,m_fx)
CVaRfr2_acc_div=matrix(0,1,m_acc)
for (i in (1:m))
{
  VaRCont_acc_div[i]=equantile(PG_acc_div[,i],1-alpha,Ns)
  CVaRCont_acc_div[i]= mean(merge(which(PG_acc_div[,i]<VaRCont_acc_div[i]),cbind(seq(1,Ns),PG_acc_div[,i]), by.x=1,by.y=1)[,2])
  if (i<=m_fx)
  {
  VaRfr1_acc_div[i]=equantile(PGfr1_acc_div[,i],1-alpha,Ns)
  CVaRfr1_acc_div[i]= mean(merge(which(PGfr1_acc_div[,i]<VaRfr1_acc_div[i]),cbind(seq(1,Ns),PGfr1_acc_div[,i]), by.x=1,by.y=1)[,2])
  }
  if (i<=m_acc)
  {  
  VaRfr2_acc_div[i]=equantile(PGfr2_acc_div[,i],1-alpha,Ns)
  CVaRfr2_acc_div[i]= mean(merge(which(PGfr2_acc_div[,i]<VaRfr2_acc_div[i]),cbind(seq(1,Ns),PGfr2_acc_div[,i]), by.x=1,by.y=1)[,2])
  }
}

VaRCont_acc_div
VaRfr1_acc_div
VaRfr2_acc_div
CVaRCont_acc_div
CVaRfr1_acc_div
CVaRfr2_acc_div


#VaR Total

VaRTotal_acc_div=equantile(PGT_acc_div,1-alpha,Ns)
CVaRTotal_acc_div= mean(merge(which(PGT_acc_div<VaRTotal_acc_div),cbind(seq(1,Ns),PGT_acc_div), by.x=1,by.y=1)[,2])
VaRTotalfr1_acc_div=equantile(PGfr1T_acc_div,1-alpha,Ns)
CVaRTotalfr1_acc_div= mean(PGfr1T_acc_div[which(PGfr1T_acc_div<VaRTotalfr1_acc_div),])
VaRTotalfr2_acc_div=equantile(PGfr2T_acc_div,1-alpha,Ns)
CVaRTotalfr2_acc_div= mean(PGfr2T_acc_div[which(PGfr2T_acc_div<VaRTotalfr2_acc_div),])


# print(cbind(VaRTotal_acc_div,sum(V0_acc_div), VaRCont_acc_div, V0_acc_div))
cbind(CVaRTotal_acc_div,sum(V0_acc_div), CVaRCont_acc_div, V0_acc_div)
cbind(VaRTotal_acc_div,VaRTotalfr1_acc_div,VaRTotalfr2_acc_div)
cbind(CVaRTotal_acc_div,CVaRTotalfr1_acc_div,CVaRTotalfr2_acc_div)

print("Valor de acciones y divisas")
print(V0_acc_div )
print("Valor en Riesgo de acciones y divisas")
print(VaRCont_acc_div)
print("Valor en Riesgo de divisas")
print(VaRTotalfr1_acc_div)
print("Valor en Riesgo de acciones")
print(VaRTotalfr2_acc_div)

print("Valor en Riesgo Condicional de acciones y divisas")
print(CVaRCont_acc_div)
print("Valor en Riesgo Condicional de divisas")
print(CVaRTotalfr1_acc_div)
print("Valor en Riesgo Condicional de acciones")
print(CVaRTotalfr2_acc_div)


[1] "Valor de acciones y divisas"
   EURUSD.X  GBPUSD.X USDMXN.X AMXL.MX GCARSOA1.MX WALMEX.MX
1: 15567.65 -15396.68 29721.75  -79600       52940     69000
[1] "Valor en Riesgo de acciones y divisas"
          [,1]      [,2]      [,3]      [,4]      [,5]      [,6]
[1,] -164.2607 -261.6068 -354.8889 -2858.266 -2664.647 -1810.567
[1] "Valor en Riesgo de divisas"
[1] -396.7038
[1] "Valor en Riesgo de acciones"
[1] -2909.172
[1] "Valor en Riesgo Condicional de acciones y divisas"
          [,1]      [,2]      [,3]      [,4]     [,5]      [,6]
[1,] -200.9584 -334.8924 -539.3833 -3557.072 -4400.62 -2313.259
[1] "Valor en Riesgo Condicional de divisas"
[1] -553.5146
[1] "Valor en Riesgo Condicional de acciones"
[1] -3561.442


### Inciso 3: Cetes

In [438]:
%%R

UsageError: %%R is a cell magic, but the cell body is empty. Did you mean the line magic %R (single %)?


### Inciso 3: Bonos M

In [ ]:
%%R

### Inciso 3: Bondes D

In [ ]:
%%R

### Inciso 4: Forwards de tipo de cambio

In [ ]:
%%R

### Inciso 4: Forwards de indice

In [ ]:
%%R

### Inciso 5: Swaps

In [446]:
%%R
#riesgo del swap
m=ncol(N)    #PASO CLAVE
X_s_sw=matrix(0,Ns,n_if[3]) #Factores de riesgo simulados con base en DeltaX_s x0*(1+Delta_Xs)   #PASO CLAVE
V_sw=matrix(0,Ns,m)
Vfr1_sw=matrix(0,Ns,m)
Vfr2_sw=matrix(0,Ns,m)
PG_sw=matrix(0,Ns,m) #Pèrdidas y ganancias
PGfr1_sw=matrix(0,Ns,m)
PGfr2_sw=matrix(0,Ns,m)
PGT_sw=matrix(0,Ns,1)
PGfr1T_sw=matrix(0,Ns,1)
PGfr2T_sw=matrix(0,Ns,1)

DeltaX_s_sw=DeltaX_s[,sum(n_if[1:2],1):sum(n_if[1:3])]    #PASO CLAVE
x0_sw=as.numeric(c(XtfwdT[1,],Xvp[1,]))   #PASO CLAVE

for (i in 1:Ns)
{
  X_s_sw[i,]=x0_sw*exp(DeltaX_s_sw[i,])
  #PASO CLAVE
  V_sw[i,]=swap(por_swT, contratos_swT, nominal_swT, X_s_sw[i,1:(n_if[3]/2)], tasafija_swT, plazocupon_swT, VTplazos_sw, X_s_sw[i,(n_if[3]/2+1):(n_if[3])],N)
  #PASO CLAVE
  Vfr1_sw[i,]=swap(por_swT, contratos_swT, nominal_swT,X_s_sw[i,1:(n_if[3]/2)], tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp[1,],N)
  #PASO CLAVE
  Vfr2_sw[i,]=swap(por_swT, contratos_swT, nominal_swT,  XtfwdT[1,], tasafija_swT, plazocupon_swT, VTplazos_sw, X_s_sw[i,(n_if[3]/2+1):(n_if[3])],N)
  #PASO CLAVE
  PG_sw[i,]=V_sw[i,]-V0_sw
  PGfr1_sw[i,]=Vfr1_sw[i,]-V0_sw
  PGfr2_sw[i,]=Vfr2_sw[i,]-V0_sw
  PGT_sw[i,]=sum(PG_sw[i,])
  PGfr1T_sw[i,]=sum(PGfr1_sw[i,])
  PGfr2T_sw[i,]=sum(PGfr2_sw[i,])
}



#VaR por posición
VaRCont_sw=matrix(0,1,m)
VaRfr1_sw=matrix(0,1,m)
VaRfr2_sw=matrix(0,1,m)
CVaRCont_sw=matrix(0,1,m)
CVaRfr1_sw=matrix(0,1,m)
CVaRfr2_sw=matrix(0,1,m)
for (i in (1:m))
{
  VaRCont_sw[i]=equantile(PG_sw[,i],1-alpha,Ns)
  VaRfr1_sw[i]=equantile(PGfr1_sw[,i],1-alpha,Ns)
  VaRfr2_sw[i]=equantile(PGfr2_sw[,i],1-alpha,Ns)
  CVaRfr1_sw[i]= mean(merge(which(PGfr1_sw[,i]<VaRfr1_sw[i]),cbind(seq(1,Ns),PGfr1_sw[,i]), by.x=1,by.y=1)[,2])
  CVaRfr2_sw[i]= mean(merge(which(PGfr2_sw[,i]<VaRfr2_sw[i]),cbind(seq(1,Ns),PGfr2_sw[,i]), by.x=1,by.y=1)[,2])
  CVaRCont_sw[i]= mean(merge(which(PG_sw[,i]<VaRCont_sw[i]),cbind(seq(1,Ns),PG_sw[,i]), by.x=1,by.y=1)[,2])
}

#VaR Total

VaRTotal_sw=equantile(PGT_sw,1-alpha,Ns)
CVaRTotal_sw= mean(merge(which(PGT_sw<VaRTotal_sw),cbind(seq(1,Ns),PGT_sw), by.x=1,by.y=1)[,2])
VaRTotalfr1_sw=equantile(PGfr1T_sw,1-alpha,Ns)
CVaRTotalfr1_sw= mean(PGfr1T_sw[which(PGfr1T_sw<VaRTotalfr1_sw),])
VaRTotalfr2_sw=equantile(PGfr2T_sw,1-alpha,Ns)
CVaRTotalfr2_sw= mean(PGfr2T_sw[which(PGfr2T_sw<VaRTotalfr2_sw),])


print(cbind(VaRTotal_sw,sum(V0_sw), VaRCont_sw, V0_sw))
print(cbind(CVaRTotal_sw,sum(V0_sw), CVaRCont_sw, V0_sw))
print(cbind(VaRTotal_sw,VaRTotalfr1_sw,VaRTotalfr2_sw))
print(cbind(CVaRTotal_sw,CVaRTotalfr1_sw,CVaRTotalfr2_sw))


     VaRTotal_sw                                                
[1,]   -4.231178 108.0531 -3.379938 -0.9049095 79.69671 28.35638
     CVaRTotal_sw                                               
[1,]    -6.319764 108.0531 -5.090684 -1.524824 79.69671 28.35638
     VaRTotal_sw VaRTotalfr1_sw VaRTotalfr2_sw
[1,]   -4.231178      -4.249618    -0.09853602
     CVaRTotal_sw CVaRTotalfr1_sw CVaRTotalfr2_sw
[1,]    -6.319764       -6.341834      -0.1893861


### Inciso 6: Opciones

In [ ]:
%%R

## Medición de riesgo, con alisado

### Generación de escenarios

In [448]:
%%R
w0 <- 0.05 #@param {type:"number"}
lambda =uniroot(function(x) w0*(1-x^(n))/(1-x)-1, c(0,0.99), tol = 1e-28)$root


#generamos la función que genera "n" escenarios con base en w0 y lambda
genera_esc=function(lamda,w0,n)
{
  p_esc=matrix(0,n,1)
  for (i in (1:n))
  {
    p_esc[i]=w0*lambda^(i-1)
  }
  p_esc
}

p_esc=genera_esc(lambda,w0,Ns)
sum(p_esc)

[1] 0.9999993


#### Inciso 1 y 2: Acciones y divisas


In [450]:
%%R
VaRCACont_acc_div=matrix(0,1,m)
VaRCAfr1_acc_div=matrix(0,1,m_fx)
VaRCAfr2_acc_div=matrix(0,1,m_acc)
CVaRCACont_acc_div=matrix(0,1,m)
CVaRCAfr1_acc_div=matrix(0,1,m_fx)
CVaRCAfr2_acc_div=matrix(0,1,m_acc)
for (i in (1:m))
{
  VaRCACont_acc_div[i]=wquantile(as.matrix(PG_acc_div[,i]), p_esc, 1-alpha)
  CVaRCACont_acc_div[i]= wcvar(as.matrix(PG_acc_div[,i]), p_esc, 1-alpha)
  if (i<=m_fx)
  {
  VaRCAfr1_acc_div[i]=wquantile(as.matrix(PGfr1_acc_div[,i]), p_esc, 1-alpha)
  CVaRCAfr1_acc_div[i]= wcvar(as.matrix(PGfr1_acc_div[,i]), p_esc, 1-alpha)
  }
  if (i<=m_acc)
  {  
  VaRCAfr2_acc_div[i]=wquantile(as.matrix(PGfr2_acc_div[,i]), p_esc, 1-alpha)
  CVaRCAfr2_acc_div[i]= wcvar(as.matrix(PGfr2_acc_div[,i]), p_esc, 1-alpha)
  }
}

VaRCACont_acc_div
VaRCAfr1_acc_div
VaRCAfr2_acc_div
CVaRCACont_acc_div
CVaRCAfr1_acc_div
CVaRCAfr2_acc_div

#VaR Total

VaRCATotal_acc_div=wquantile(PGT_acc_div,p_esc, 1-alpha)
CVaRCATotal_acc_div= wcvar(PGT_acc_div,p_esc,1-alpha)
VaRCATotalfr1_acc_div=wquantile(PGfr1T_acc_div,p_esc,1-alpha)
CVaRCATotalfr1_acc_div= wcvar(PGfr1T_acc_div,p_esc,1-alpha)
VaRCATotalfr2_acc_div=wquantile(PGfr2T_acc_div,p_esc,1-alpha)
CVaRCATotalfr2_acc_div= wcvar(PGfr2T_acc_div,p_esc,1-alpha)

print("Valor de acciones y divisas")
print(V0_acc_div )
print("Valor en Riesgo de acciones y divisas")
print(VaRCACont_acc_div)
print("Valor en Riesgo de divisas")
print(VaRCATotalfr1_acc_div)
print("Valor en Riesgo de acciones")
print(VaRCATotalfr2_acc_div)

print("Valor en Riesgo Condicional de acciones y divisas")
print(CVaRCACont_acc_div)
print("Valor en Riesgo Condicional de divisas")
print(CVaRCATotalfr1_acc_div)
print("Valor en Riesgo Condicional de acciones")
print(CVaRCATotalfr2_acc_div)

[1] "Valor de acciones y divisas"
   EURUSD.X  GBPUSD.X USDMXN.X AMXL.MX GCARSOA1.MX WALMEX.MX
1: 15567.65 -15396.68 29721.75  -79600       52940     69000
[1] "Valor en Riesgo de acciones y divisas"
          [,1]      [,2]
[1,] -200.4597 -326.4396
[1] "Valor en Riesgo de divisas"
[1] -455.6402
[1] "Valor en Riesgo de acciones"
[1] -3997.393
[1] "Valor en Riesgo Condicional de acciones y divisas"
          [,1]      [,2]
[1,] -247.2496 -398.2374
[1] "Valor en Riesgo Condicional de divisas"
[1] -659.7572
[1] "Valor en Riesgo Condicional de acciones"
[1] -4639.764


#### Inciso 3: Cetes

In [ ]:
%%R

#### Inciso 3: Bonos M

In [ ]:
%%R

#### Inciso 3: Bondes D

In [ ]:
%%R

#### Inciso 4: Forward de tipo de cambio

In [ ]:
%%R

#### Inciso 4: Forwards de indice

In [ ]:
%%R

#### Inciso 5: Swaps

In [ ]:
%%R

#### Inciso 6: Opciones

In [ ]:
%%R

## Riesgo integral

In [451]:
%%R
#Medición de riesgo por factor de riesgo de todo el portafolios
#Acciones
#1. Acciones 
PGPort_ACC=PGfr2T_acc_div + PGfr3T_fwind #Pérdidas y ganancias
VaRPort_ACC=equantile(PGPort_ACC,1-alpha,Ns) #VaR
CVaRPort_ACC= mean(PGPort_ACC[which(PGPort_ACC<VaRPort_ACC)]) #CVaR


#Tasa de Interés
#1. Dado que swaps y bondes son de tasa de interés usaremos PGT_bd y PGT_sw
#2. Para futuros usaremos PGfr1T_fwtdc y PGfr2T_fwtdc
PGPort_TI=PGT_bd+PGT_sw+PGfr1T_oir +PGfr2T_oir + PGfr1T_fwind +PGfr2T_fwind + PGfr1T_fwtdc + PGfr2T_fwtdc#Pérdidas y ganancias
VaRPort_TI=equantile(PGPort_TI,1-alpha,Ns) #VaR
CVaRPort_TI= mean(PGPort_TI[which(PGPort_TI<VaRPort_TI)]) #CVaR

#Tipo de cambio
#1. Dado que swaps y bondes son de tasa de interés no usamos nada
#2. Para futuros usamos sólo PGfr3T_fwtdc
PGPort_TDC=PGfr1T_acc_div + PGfr3T_fwtdc  #Pérdidas y ganancias
VaRPort_TDC=equantile(PGPort_TDC,1-alpha,Ns) #VaR
CVaRPort_TDC= mean(PGPort_TDC[which(PGPort_TDC<VaRPort_TDC)]) #CVaR

#Volatilidad
#1. Sólo aplica la volatilidad de Opciones de tasa de interés

PGPort_VOL=PGfr3T_oir  #Pérdidas y ganancias
VaRPort_VOL=equantile(PGPort_VOL,1-alpha,Ns) #VaR
CVaRPort_VOL= mean(PGPort_VOL[which(PGPort_VOL<VaRPort_VOL)]) #CVaR


#Medición de riesgo de todo el portafolios
#Sumar todos los PGT de todos los instrumentos

PGT_Port=PGPort_ACC+PGPort_TI+PGPort_TDC+PGPort_VOL
VaRTotal_Port=equantile(PGT_Port,1-alpha,Ns) #VaR
CVaRTotal_Port= mean(PGT_Port[which(PGT_Port<VaRTotal_Port)]) #CVaR

R[write to console]: Error in withVisible({ : object 'PGfr3T_fwind' not found




Error in withVisible({ : object 'PGfr3T_fwind' not found


RInterpreterError: ignored